In [1]:
!pip install BackTranslation

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 6.0 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17395 sha256=cc3b2a2db7b2bab679aab77460e191be4add4cbd768d497ad980ee49955246cb
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling ch

In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset

dataset = load_dataset("krishan-CSE/Davidson_Hate_Speeh_Original")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
from BackTranslation import BackTranslation
trans = BackTranslation(url=[
      'translate.google.com',
      'translate.google.co.kr',
    ], proxies={'http': '127.0.0.1:1234', 'http://host.name': '127.0.0.1:4012'})

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [11]:
import pandas as pd


# Define the column names
columns = ['text', 'labels']

# Create an empty DataFrame with the specified columns
df = pd.DataFrame(columns=columns)
df_ref= pd.DataFrame(columns=columns)


In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'count', 'hate_speech', 'offensive_language', 'neither', 'class', 'tweet'],
        num_rows: 24783
    })
})

In [12]:
from tqdm.auto import tqdm

In [ ]:
for example in tqdm(dataset['train']):
    # Access example fields based on the dataset's structure
    text = example['tweet']  # Assuming 'text' is a field in your dataset
    label = example['class']  # Assuming 'labels' is a field in your dataset
    if label==0:
      result = trans.translate(text, src='en', tmp = 'fr',sleeping=1)
      new_row = {'text': result.result_text, 'labels': label}  # Define the new row as a dictionary
      # Convert the new row into a DataFrame
      new_df = pd.DataFrame([new_row])
      # Concatenate the original DataFrame with the new DataFrame
      df = pd.concat([df, new_df], ignore_index=True)
      #df = df.append(new_row, ignore_index=True)  # Append the new row to the DataFrame
      df_ref= pd.concat([df_ref,pd.DataFrame([{'text': text, 'labels': label}])], ignore_index=True)

  0%|          | 0/24783 [00:00<?, ?it/s]

In [ ]:
df.to_csv("output.csv", index=False)
df_ref.to_csv("output_red.csv",index=False)